In [1]:
# do not touch this cell 
%load_ext jupyter_record


In [2]:
%%git_commit

import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack,vstack
import numpy as np


In [3]:
%%git_commit

training = pd.read_csv('../archive/Combined_Flights_2018.csv').sample(100000)
testing = pd.read_csv('../archive/Combined_Flights_2019.csv').sample(100000)




In [5]:
%%git_commit

training = training[training['Diverted'] == 0]
testing = testing[testing['Cancelled'] == 0]



In [7]:
%%git_commit
print(training.keys())

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [8]:
%%git_commit
training = training[['DayOfWeek', 'Airline', 'CRSDepTime', 'Origin', 'Dest', 'Distance', 'ArrDel15']]
testing = testing[['DayOfWeek', 'Airline', 'CRSDepTime', 'Origin', 'Dest', 'Distance', 'ArrDel15']]



In [9]:
%%git_commit

training.dropna()
training = training.reset_index(drop=True)
testing.dropna()
testing = testing.reset_index(drop=True)


In [11]:
%%git_commit
training = training.rename(columns = {'DayOfWeek' : 'Weekend'})
training['Weekend'] = (training['Weekend'] > 5).astype(int)
testing = testing.rename(columns = {'DayOfWeek' : 'Weekend'})
testing['Weekend'] = (testing['Weekend'] > 5).astype(int)

In [13]:
%%git_commit
training['Airline'].nunique()

28

In [14]:
%%git_commit
carrier_df = training[['Airline','ArrDel15']].groupby('Airline').sum().sort_values(by='ArrDel15',ascending=False)
carrier_df['CARRIER_cat'] = pd.qcut(carrier_df['ArrDel15'], 28, labels = False)
carrier_df


,ArrDel15,CARRIER_cat
Airline,,
Southwest Airlines Co.,4637.0,27
United Air Lines Inc.,2083.0,26
SkyWest Airlines Inc.,1716.0,25
JetBlue Airways,1398.0,24
American Airlines Inc.,1364.0,23
Delta Air Lines Inc.,996.0,22
Republic Airlines,656.0,21
ExpressJet Airlines Inc.,600.0,20
Frontier Airlines Inc.,572.0,19


In [15]:
%%git_commit
data_carrier = carrier_df.loc[training['Airline']].reset_index()
training['CARRIER_cat'] = data_carrier['CARRIER_cat']


In [16]:
%%git_commit
data_carrier = carrier_df.loc[testing['Airline']].reset_index()
testing['CARRIER_cat'] = data_carrier['CARRIER_cat']

In [17]:
%%git_commit
training['CRSDepTime'] = training['CRSDepTime']//100
testing['CRSDepTime'] = testing['CRSDepTime']//100

In [19]:
%%git_commit
training['CRSDepTime'].nunique()


24

In [21]:
%%git_commit
time_blk_df = training[['CRSDepTime','ArrDel15']].groupby('CRSDepTime').sum().sort_values(by='ArrDel15',ascending=False)
time_blk_df['TIME_cat'] = pd.qcut(time_blk_df['ArrDel15'], 19, labels = False)
time_blk_df

,ArrDel15,TIME_cat
CRSDepTime,,
17,1694.0,18
18,1510.0,18
19,1438.0,17
16,1328.0,16
15,1313.0,15
14,1281.0,14
20,1209.0,14
12,1141.0,13
13,1136.0,12


In [22]:
%%git_commit
data_time = time_blk_df.loc[training['CRSDepTime']].reset_index()
training['DEP_TIME_cat'] = data_time['TIME_cat']

In [23]:
%%git_commit
data_time = time_blk_df.loc[testing['CRSDepTime']].reset_index()
testing['DEP_TIME_cat'] = data_time['TIME_cat']

In [29]:
%%git_commit

training['Origin'].nunique()


366

In [35]:
%%git_commit
origin_df = training[['Origin','ArrDel15']].groupby('Origin').sum().sort_values(by='ArrDel15',ascending=False)
origin_df['ORIGIN_cat'] = pd.qcut(origin_df['ArrDel15'], 10, labels = False)
origin_df

,ArrDel15,ORIGIN_cat
Origin,,
ORD,1177.0,9
DEN,820.0,9
EWR,689.0,9
ATL,639.0,9
SFO,588.0,9
...,...,...
DRT,0.0,0
HVN,0.0,0
IMT,0.0,0


In [39]:
%%git_commit
for key in testing['Origin'].unique():
    if key not in origin_df.index:
        df.loc[len(df.index)] = ['Amy', 89, 93]

WYS
XWA
PAE
ATY
APN
BFM
PIR
AKN


In [36]:
%%git_commit
data_origin = origin_df.loc[training['Origin']].reset_index()
training['ORIGIN_cat'] = data_origin['ORIGIN_cat']

data_origin = origin_df.loc[testing['Origin']].reset_index()
testing['ORIGIN_cat'] = data_origin['ORIGIN_cat']

KeyError: "['WYS', 'XWA', 'PAE', 'ATY', 'APN', 'BFM', 'PIR', 'AKN'] not in index"

In [ ]:
%%git_commit
